In [1]:
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1753856736049_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = spark.read.parquet("s3://nyc-housing-complaints-g4/final/master_complaints_table/part-00000-0f889d5f-ac8a-43f6-a9bf-fc4787c6de99-c000.snappy.parquet")
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+-------------------+-------------------+--------------------+------------+--------------------+------+----------+-----------+------------+--------+--------------------+-----------------+------------------+----------+---------+-------+-------+--------------------+---------+---------+--------------------+-------+--------+-------+-------+--------+----------+---------+---------+----------+----------+---------+--------+--------------+---------------+------------+
|Unique_Key|        Created_Date|         Closed_Date| created_date_stand|  closed_date_stand|      complaint_type|  Descriptor|  complaint_category|Status|validation|    borough|Incident_Zip|    City|        full_address|         Latitude|         Longitude|       bbl|tax_block|tax_lot|landuse|    landuse_category|bldgclass|ownertype|           ownername|lotarea|bldgarea|resarea|comarea|unitsres|unitstotal|numfloors|yearbuilt|yearalter1|yearalter2|zonedist1|overlay1|latitude_pl

In [3]:
# Row and column count
row_count = df.count()

row_count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11136112

In [4]:
col_count = len(df.columns)
col_count



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

39

In [5]:
from pyspark.sql.functions import col, sum
#null count
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+-----------+------------------+-----------------+--------------+----------+------------------+------+----------+-------+------------+------+------------+--------+---------+-------+---------+-------+-------+----------------+---------+---------+---------+-------+--------+-------+-------+--------+----------+---------+---------+----------+----------+---------+--------+--------------+---------------+------------+
|Unique_Key|Created_Date|Closed_Date|created_date_stand|closed_date_stand|complaint_type|Descriptor|complaint_category|Status|validation|borough|Incident_Zip|  City|full_address|Latitude|Longitude|    bbl|tax_block|tax_lot|landuse|landuse_category|bldgclass|ownertype|ownername|lotarea|bldgarea|resarea|comarea|unitsres|unitstotal|numfloors|yearbuilt|yearalter1|yearalter2|zonedist1|overlay1|latitude_pluto|longitude_pluto|bbl_standard|
+----------+------------+-----------+------------------+-----------------+--------------+----------+------------------+------+

In [6]:
from pyspark.sql import functions as F

# Calculate null counts (same as before)
null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])


expr = "stack({0}, {1}) as (Column, Null_Count)".format(
    len(df.columns),
    ",".join(["'{0}', {0}".format(c) for c in df.columns])
)

null_counts_rowwise = null_counts.selectExpr(expr)
null_counts_rowwise.show(n=5000,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------+
|Column            |Null_Count|
+------------------+----------+
|Unique_Key        |0         |
|Created_Date      |0         |
|Closed_Date       |137992    |
|created_date_stand|0         |
|closed_date_stand |137992    |
|complaint_type    |0         |
|Descriptor        |7098      |
|complaint_category|0         |
|Status            |0         |
|validation        |0         |
|borough           |12        |
|Incident_Zip      |260501    |
|City              |259741    |
|full_address      |0         |
|Latitude          |260441    |
|Longitude         |260441    |
|bbl               |1847717   |
|tax_block         |1892015   |
|tax_lot           |1892015   |
|landuse           |1892015   |
|landuse_category  |1892015   |
|bldgclass         |1892015   |
|ownertype         |10141604  |
|ownername         |1892073   |
|lotarea           |1892039   |
|bldgarea          |1892015   |
|resarea           |1946641   |
|comarea           |1946641   |
|unitsre

In [7]:
# List of columns to drop
drop_columns = [
    'created_date_stand', 'closed_date_stand', 'validation', 'bbl', 'tax_block', 
    'tax_lot', 'landuse', 'landuse_category', 'bldgclass', 'ownertype', 'ownername', 
    'lotarea', 'bldgarea', 'resarea', 'comarea', 'unitsres', 'unitstotal', 
    'numfloors', 'yearbuilt', 'yearalter1', 'yearalter2', 'zonedist1', 
    'overlay1', 'latitude_pluto', 'longitude_pluto', 'bbl_standard'
]

# Drop columns from DataFrame
df_cleaned = df.drop(*drop_columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:

# Verify columns after dropping
df_cleaned.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Unique_Key: string (nullable = true)
 |-- Created_Date: string (nullable = true)
 |-- Closed_Date: string (nullable = true)
 |-- complaint_type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- complaint_category: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- Incident_Zip: string (nullable = true)
 |-- City: string (nullable = true)
 |-- full_address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

In [9]:
from pyspark.sql.functions import to_timestamp

df_cleaned = df_cleaned \
    .withColumn("Created_Date", to_timestamp("Created_Date", "MM/dd/yyyy HH:mm:ss")) \
    .withColumn("Closed_Date", to_timestamp("Closed_Date", "MM/dd/yyyy HH:mm:ss"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.sql.functions import col

df_cleaned = df_cleaned \
    .withColumn("Latitude", col("Latitude").cast("double")) \
    .withColumn("Longitude", col("Longitude").cast("double"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_cleaned.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Unique_Key: string (nullable = true)
 |-- Created_Date: timestamp (nullable = true)
 |-- Closed_Date: timestamp (nullable = true)
 |-- complaint_type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- complaint_category: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- Incident_Zip: string (nullable = true)
 |-- City: string (nullable = true)
 |-- full_address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)

In [12]:
from pyspark.sql.functions import col, sum

df_cleaned.select([
    sum(col(c).isNull().cast("int")).alias(c) 
    for c in ["Latitude", "Longitude", "Created_Date", "Closed_Date"]
]).show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+------------+-----------+
|Latitude|Longitude|Created_Date|Closed_Date|
+--------+---------+------------+-----------+
|  260441|   260441|           0|     137992|
+--------+---------+------------+-----------+

In [13]:
from pyspark.sql import functions as F

# Fill missing Latitude/Longitude with average per borough
avg_coords = df_cleaned.groupBy("borough") \
    .agg(F.avg("Latitude").alias("avg_lat"), F.avg("Longitude").alias("avg_lon"))

df_cleaned = df_cleaned.join(avg_coords, on="borough", how="left") \
    .withColumn("Latitude", F.coalesce("Latitude", "avg_lat")) \
    .withColumn("Longitude", F.coalesce("Longitude", "avg_lon")) \
    .drop("avg_lat", "avg_lon")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.sql.functions import col, sum

df_cleaned.select([
    sum(col(c).isNull().cast("int")).alias(c) 
    for c in ["Latitude", "Longitude", "Created_Date", "Closed_Date"]
]).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+------------+-----------+
|Latitude|Longitude|Created_Date|Closed_Date|
+--------+---------+------------+-----------+
|      12|       12|           0|     137992|
+--------+---------+------------+-----------+

In [15]:
# checking for invalid numerical values
invalid_lat = df_cleaned.filter((col("Latitude") < -90) | (col("Latitude") > 90))
invalid_lon = df_cleaned.filter((col("Longitude") < -180) | (col("Longitude") > 180))

print("Invalid Latitude Count:", invalid_lat.count())
print("Invalid Longitude Count:", invalid_lon.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Invalid Latitude Count: 0
Invalid Longitude Count: 0

In [16]:
# Count invalid (null) dates
df_cleaned.select([
    sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") 
    for c in ["Created_Date", "Closed_Date"]
]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Created_Date_nulls|Closed_Date_nulls|
+------------------+-----------------+
|                 0|           137992|
+------------------+-----------------+

In [17]:
# Check for closed dates earlier than created dates
invalid_dates = df_cleaned.filter(col("Closed_Date") < col("Created_Date"))
print("Closed_Date earlier than Created_Date:", invalid_dates.count())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Closed_Date earlier than Created_Date: 135790

In [18]:
#  Replace Closed_Date nulls with "Open"
# If the complaint is still open, it makes sense to mark Closed_Date as "Open" or a placeholder
from pyspark.sql.functions import when, lit

df_cleaned = df_cleaned.withColumn(
    "Closed_Date",
    when(col("Closed_Date").isNull(), lit("Open")).otherwise(col("Closed_Date"))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.sql.functions import col, sum

# Re-check if any nulls still exist in Closed_Date
df_cleaned.select(
    sum(col("Closed_Date").isNull().cast("int")).alias("Closed_Date_nulls")
).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|Closed_Date_nulls|
+-----------------+
|                0|
+-----------------+

In [20]:
# View some rows with "Open"
df_cleaned.filter(col("Closed_Date") == "Open").show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-------------------+-----------+--------------------+----------+------------------+------+------------+--------+--------------------+----------------+------------------+
| borough|Unique_Key|       Created_Date|Closed_Date|      complaint_type|Descriptor|complaint_category|Status|Incident_Zip|    City|        full_address|        Latitude|         Longitude|
+--------+----------+-------------------+-----------+--------------------+----------+------------------+------+------------+--------+--------------------+----------------+------------------+
|BROOKLYN|  20228670|2011-04-13 12:00:00|       Open|            electric|  LIGHTING|  ELECTRICAL_ISSUE|  Open|       11220|BROOKLYN|740 59 street 59 ...|40.6368193821522|-74.01020019903407|
|BROOKLYN|  20230671|2011-04-13 12:00:00|       Open|     paint - plaster|   CEILING|     PAINT_PLASTER|  Open|       11220|BROOKLYN|740 59 street 59 ...|40.6368193821522|-74.01020019903407|
|BROOKLYN|  20201537|2011-04-08 12:00:00|    

In [21]:
from pyspark.sql.functions import col, sum

df_cleaned.select([
    sum(col(c).isNull().cast("int")).alias(c) 
    for c in ["Latitude", "Longitude", "Created_Date", "Closed_Date"]
]).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+------------+-----------+
|Latitude|Longitude|Created_Date|Closed_Date|
+--------+---------+------------+-----------+
|      12|       12|           0|          0|
+--------+---------+------------+-----------+

In [24]:
df_cleaned.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .parquet("s3://bigdata-project-handlemissing/mater_table_after_nulls/")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…